### Sample Keras [Implementation](https://www.kaggle.com/nikhithn/sample-keras-nn-implementation/code) 

In [213]:
import numpy
import pandas
import matplotlib.pyplot as plt
import subprocess
from tqdm import tqdm
from keras_tqdm import TQDMNotebookCallback
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
import csv
import sklearn

In [105]:
pandas.set_option('display.max_columns', 100)
pandas.set_option('display.max_rows', 100)
pandas.set_option('display.max_colwidth', -1)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [4]:
trainingOrderItems = pandas.read_csv("data/order_products__train.csv")
priorOrderItems = pandas.read_csv("data/order_products__prior.csv")

orders = pandas.read_csv("data/orders.csv")

aisles = pandas.read_csv("data/aisles.csv", )
departments = pandas.read_csv("data/departments.csv")
products = pandas.read_csv("data/products.csv")


In [6]:
priorOrderItems.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [16]:
productGroup = priorOrderItems.groupby("product_id")
productReorders = productGroup['order_id'].aggregate(numpy.size).to_frame()
productReorders['reordered'] = productGroup['reordered'].aggregate(numpy.mean)

del productGroup

In [17]:
orderGroup = priorOrderItems.groupby("order_id").aggregate(numpy.max)['add_to_cart_order'].to_frame()
priorOrderItems = pandas.merge(priorOrderItems,orderGroup.rename(columns={'add_to_cart_order':'basket_size'}),left_on='order_id', right_index=True)
priorOrderItems.head()

,order_id,product_id,add_to_cart_order,reordered,basket_size
0,2,33120,1,1,9
1,2,28985,2,1,9
2,2,9327,3,0,9
3,2,45918,4,1,9
4,2,30035,5,0,9


In [32]:
# Let's now look at products by user
orderItems = pandas.merge(orders,priorOrderItems, on='order_id')
orderItems['user_product_id'] = 1000000*orderItems['user_id']+orderItems['product_id']

userProducts = orderItems[orderItems['eval_set']=='prior']
userProducts = userProducts[['user_id','order_dow','order_hour_of_day','days_since_prior_order',
                           'product_id','add_to_cart_order', 'user_product_id', 'basket_size', 'order_number', 'order_id']]
userProducts['orders'] = userProducts['user_id']
userAvgProducts = userProducts.groupby('user_product_id').aggregate({'user_id':'mean','order_dow':'mean','order_hour_of_day':'mean',
                                                   'days_since_prior_order':'mean', 'orders':'count',
                                                   'product_id':'mean','add_to_cart_order':'mean', 'basket_size':'mean'})
del orderItems

In [33]:
userAvgProducts.head()

,user_id,order_dow,order_hour_of_day,days_since_prior_order,orders,product_id,add_to_cart_order,basket_size
user_product_id,,,,,,,,
1000196,1,2.500000,10.300000,19.555556,10,196,1.400000,5.900000
1010258,1,2.555556,10.555556,19.555556,9,10258,3.333333,6.000000
1010326,1,4.000000,15.000000,28.000000,1,10326,5.000000,8.000000
1012427,1,2.500000,10.300000,19.555556,10,12427,3.300000,5.900000
1013032,1,2.666667,8.000000,21.666667,3,13032,6.333333,6.666667


In [34]:
lastOrderId = userProducts[userProducts.groupby(['user_product_id'])['order_number'].transform(max) == userProducts['order_number']]
lastOrderId = lastOrderId[['user_product_id','order_id']]
lastOrderId.rename(columns={'order_id':'last_order_id'}, inplace=True)
userAvgProducts = pandas.merge(userAvgProducts, lastOrderId, right_on='user_product_id', left_index=True)
userAvgProducts.set_index('user_product_id', inplace=True)
userAvgProducts.head()

,user_id,order_dow,order_hour_of_day,days_since_prior_order,orders,product_id,add_to_cart_order,basket_size,last_order_id
user_product_id,,,,,,,,,
1000196,1,2.500000,10.300000,19.555556,10,196,1.400000,5.900000,2550362
1010258,1,2.555556,10.555556,19.555556,9,10258,3.333333,6.000000,2550362
1010326,1,4.000000,15.000000,28.000000,1,10326,5.000000,8.000000,431534
1012427,1,2.500000,10.300000,19.555556,10,12427,3.300000,5.900000,2550362
1013032,1,2.666667,8.000000,21.666667,3,13032,6.333333,6.666667,2550362


In [35]:
user_distinct_products = userAvgProducts.groupby('user_id').count()['product_id'].to_frame()
user_distinct_products['user_reordered_products'] = (userAvgProducts[userAvgProducts['orders']>1]
                                                         .groupby('user_id')
                                                         .count()['product_id'])
user_distinct_products.rename(columns={'product_id':'distinct_products'},inplace=True)
user_distinct_products.head()

,distinct_products,user_reordered_products
user_id,,
1,18,10.0
2,102,37.0
3,33,19.0
4,17,1.0
5,23,8.0


In [ ]:
userAvgProducts = pandas.merge(user_distinct_products.reset_index(),
                               userAvgProducts.reset_index(),
                               on='user_id')

In [38]:
userAvgProducts['user_reorder_rate'] = userAvgProducts['user_reordered_products']/userAvgProducts['distinct_products']
userAvgProducts.head()

,user_id,distinct_products,user_reordered_products,user_product_id,order_dow,order_hour_of_day,days_since_prior_order,orders,product_id,add_to_cart_order,basket_size,last_order_id,user_reorder_rate
0,1,18,10.0,1000196,2.500000,10.300000,19.555556,10,196,1.400000,5.900000,2550362,0.555556
1,1,18,10.0,1010258,2.555556,10.555556,19.555556,9,10258,3.333333,6.000000,2550362,0.555556
2,1,18,10.0,1010326,4.000000,15.000000,28.000000,1,10326,5.000000,8.000000,431534,0.555556
3,1,18,10.0,1012427,2.500000,10.300000,19.555556,10,12427,3.300000,5.900000,2550362,0.555556
4,1,18,10.0,1013032,2.666667,8.000000,21.666667,3,13032,6.333333,6.666667,2550362,0.555556


In [156]:
userAvgProducts.shape

(13307953, 13)

In [167]:
features = userAvgProducts.set_index('user_product_id')
features.head()

,user_id,distinct_products,user_reordered_products,order_dow,order_hour_of_day,days_since_prior_order,orders,product_id,add_to_cart_order,basket_size,last_order_id,user_reorder_rate
user_product_id,,,,,,,,,,,,
1000196,1,18,10.0,2.500000,10.300000,19.555556,10,196,1.400000,5.900000,2550362,0.555556
1010258,1,18,10.0,2.555556,10.555556,19.555556,9,10258,3.333333,6.000000,2550362,0.555556
1010326,1,18,10.0,4.000000,15.000000,28.000000,1,10326,5.000000,8.000000,431534,0.555556
1012427,1,18,10.0,2.500000,10.300000,19.555556,10,12427,3.300000,5.900000,2550362,0.555556
1013032,1,18,10.0,2.666667,8.000000,21.666667,3,13032,6.333333,6.666667,2550362,0.555556


In [168]:
features.columns = ['user_id','user_distinct_products','user_reordered_products',
                    'user_product_dow','user_product_hod',
                    'user_product_dsp','user_product_orders','product_id',
                    'user_product_addCart', 'user_avg_basket_size',
                    'user_product_last_order_id','user_reorder_rate']

In [169]:
print(features.shape)
features.head()

(13307953, 12)


,user_id,user_distinct_products,user_reordered_products,user_product_dow,user_product_hod,user_product_dsp,user_product_orders,product_id,user_product_addCart,user_avg_basket_size,user_product_last_order_id,user_reorder_rate
user_product_id,,,,,,,,,,,,
1000196,1,18,10.0,2.500000,10.300000,19.555556,10,196,1.400000,5.900000,2550362,0.555556
1010258,1,18,10.0,2.555556,10.555556,19.555556,9,10258,3.333333,6.000000,2550362,0.555556
1010326,1,18,10.0,4.000000,15.000000,28.000000,1,10326,5.000000,8.000000,431534,0.555556
1012427,1,18,10.0,2.500000,10.300000,19.555556,10,12427,3.300000,5.900000,2550362,0.555556
1013032,1,18,10.0,2.666667,8.000000,21.666667,3,13032,6.333333,6.666667,2550362,0.555556


In [170]:
userOrders = orders[orders['eval_set']=='prior'].groupby('user_id').aggregate({'order_number':'max'})

In [171]:
print(userOrders.shape)
userOrders.head()

(206209, 1)


,order_number
user_id,
1,10
2,14
3,12
4,5
5,4


In [174]:
features = userOrders.reset_index().merge(features.reset_index(), on='user_id')
print(features_test.shape)
features.head()

(13307953, 14)


,user_id,order_number,user_product_id,user_distinct_products,user_reordered_products,user_product_dow,user_product_hod,user_product_dsp,user_product_orders,product_id,user_product_addCart,user_avg_basket_size,user_product_last_order_id,user_reorder_rate
0,1,10,1000196,18,10.0,2.500000,10.300000,19.555556,10,196,1.400000,5.900000,2550362,0.555556
1,1,10,1010258,18,10.0,2.555556,10.555556,19.555556,9,10258,3.333333,6.000000,2550362,0.555556
2,1,10,1010326,18,10.0,4.000000,15.000000,28.000000,1,10326,5.000000,8.000000,431534,0.555556
3,1,10,1012427,18,10.0,2.500000,10.300000,19.555556,10,12427,3.300000,5.900000,2550362,0.555556
4,1,10,1013032,18,10.0,2.666667,8.000000,21.666667,3,13032,6.333333,6.666667,2550362,0.555556


In [175]:
features['order_number'] = features['user_product_orders']/features['order_number']
features.rename(columns={'order_number': 'user_product_reorder_rate'}, inplace=True)
features = features.set_index('user_product_id')

In [176]:
features=pandas.merge(features,productReorders, left_on='product_id',right_index=True)
features.rename(columns={'order_id':'product_total_orders',
                         'reordered':'product_reorder_rate'},
                inplace=True)
features['product_reorders'] = features['product_reorder_rate'] * features['product_total_orders']
features.head()

,user_id,user_product_reorder_rate,user_distinct_products,user_reordered_products,user_product_dow,user_product_hod,user_product_dsp,user_product_orders,product_id,user_product_addCart,user_avg_basket_size,user_product_last_order_id,user_reorder_rate,product_total_orders,product_reorder_rate,product_reorders
user_product_id,,,,,,,,,,,,,,,,
1000196,1,1.000000,18,10.0,2.5,10.300000,19.555556,10,196,1.400000,5.900000,2550362,0.555556,35791,0.77648,27791.0
15000196,15,0.227273,13,9.0,1.4,11.000000,8.600000,5,196,2.200000,2.800000,487368,0.692308,35791,0.77648,27791.0
19000196,19,0.333333,133,39.0,5.0,14.333333,6.666667,3,196,6.333333,31.333333,532817,0.293233,35791,0.77648,27791.0
21000196,21,0.030303,102,30.0,3.0,10.000000,6.000000,1,196,2.000000,7.000000,1573906,0.294118,35791,0.77648,27791.0
31000196,31,0.100000,190,66.0,3.0,9.500000,7.500000,2,196,15.500000,27.500000,2231262,0.347368,35791,0.77648,27791.0


In [177]:
# features = pandas.merge(features, products, on='product_id')
# features.drop('product_name',axis=1,inplace=True)

In [178]:
user_order_group = (orders[orders['eval_set']=='prior']
                    .groupby('user_id')
                    .agg({'order_id':'count',
                          'days_since_prior_order':'mean'}))
user_order_group.columns = ['user_orders','user_order_dsp']

In [179]:
features = features.reset_index().merge(user_order_group.reset_index(),on='user_id')
features = features.set_index('user_product_id')
features['user_total_products'] = features['user_avg_basket_size']*features['user_orders']

In [180]:
features.head()

,user_id,user_product_reorder_rate,user_distinct_products,user_reordered_products,user_product_dow,user_product_hod,user_product_dsp,user_product_orders,product_id,user_product_addCart,user_avg_basket_size,user_product_last_order_id,user_reorder_rate,product_total_orders,product_reorder_rate,product_reorders,user_orders,user_order_dsp,user_total_products
user_product_id,,,,,,,,,,,,,,,,,,,
1000196,1,1.0,18,10.0,2.500000,10.300000,19.555556,10,196,1.400000,5.900000,2550362,0.555556,35791,0.776480,27791.0,10,19.555556,59.000000
1010258,1,0.9,18,10.0,2.555556,10.555556,19.555556,9,10258,3.333333,6.000000,2550362,0.555556,1946,0.713772,1389.0,10,19.555556,60.000000
1010326,1,0.1,18,10.0,4.000000,15.000000,28.000000,1,10326,5.000000,8.000000,431534,0.555556,5526,0.652009,3603.0,10,19.555556,80.000000
1012427,1,1.0,18,10.0,2.500000,10.300000,19.555556,10,12427,3.300000,5.900000,2550362,0.555556,6476,0.740735,4797.0,10,19.555556,59.000000
1013032,1,0.3,18,10.0,2.666667,8.000000,21.666667,3,13032,6.333333,6.666667,2550362,0.555556,3751,0.657158,2465.0,10,19.555556,66.666667


In [181]:
# Ready the inputs into model
train_orders = orders[orders['eval_set']=='train']
test_orders = orders[orders['eval_set']=='test']

In [182]:
test_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,test,13,5,15,11.0
44,329954,4,test,6,3,12,30.0
53,1528013,6,test,4,3,16,22.0
96,1376945,11,test,8,6,11,8.0
102,1356845,12,test,6,1,20,30.0


In [183]:
test_orders.shape

(75000, 7)

In [184]:
train_features = pandas.merge(train_orders, features.reset_index(), on='user_id')
test_features = pandas.merge(test_orders, features.reset_index(), on='user_id')

In [185]:
test_features.shape

(4833292, 26)

In [186]:
test_features.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,user_product_id,user_product_reorder_rate,user_distinct_products,user_reordered_products,user_product_dow,user_product_hod,user_product_dsp,user_product_orders,product_id,user_product_addCart,user_avg_basket_size,user_product_last_order_id,user_reorder_rate,product_total_orders,product_reorder_rate,product_reorders,user_orders,user_order_dsp,user_total_products
0,2774568,3,test,13,5,15,11.0,3016797,0.250000,33,19.0,0.333333,15.000000,7.000000,3,16797,4.000000,8.000000,3160850,0.575758,142951,0.698155,99802.0,12,12.090909,96.000000
1,2774568,3,test,13,5,15,11.0,3047766,0.750000,33,19.0,0.888889,16.111111,12.125000,9,47766,3.777778,7.222222,1402502,0.575758,176815,0.758103,134044.0,12,12.090909,86.666667
2,2774568,3,test,13,5,15,11.0,3000248,0.083333,33,19.0,3.000000,19.000000,9.000000,1,248,3.000000,9.000000,444309,0.575758,6371,0.400251,2550.0,12,12.090909,108.000000
3,2774568,3,test,13,5,15,11.0,3001005,0.083333,33,19.0,3.000000,16.000000,17.000000,1,1005,5.000000,6.000000,676467,0.575758,463,0.440605,204.0,12,12.090909,72.000000
4,2774568,3,test,13,5,15,11.0,3001819,0.250000,33,19.0,0.666667,16.333333,11.333333,3,1819,2.666667,7.333333,1839752,0.575758,2424,0.492162,1193.0,12,12.090909,88.000000


In [187]:
test_features.shape

(4833292, 26)

In [98]:
# del train_orders
# del test_orders
# del features

In [190]:
train_features['diff_order_hod'] = abs(train_features['user_product_hod']-train_features['order_hour_of_day']).map(lambda x: min(x, 24-x))
train_features['ratio_dsp'] = train_features['user_product_dsp']/train_features['days_since_prior_order']
train_features['diff_dow'] = abs(train_features['user_product_dow']-train_features['order_dow']).map(lambda x: min (x, 7-x))
train_features['user_product_orders_since_last'] = train_features['user_orders'] - train_features['user_product_last_order_id'].map(orders.order_number)
train_features['user_product_hour_vs_last'] = abs(train_features['order_hour_of_day'] - train_features['user_product_last_order_id'].map(orders.order_hour_of_day)                                                  ).map(lambda x: min(x, 24-x))

test_features['diff_order_hod'] = abs(test_features['user_product_hod']-test_features['order_hour_of_day']).map(lambda x: min(x, 24-x))
test_features['ratio_dsp'] = test_features['user_product_dsp']/test_features['days_since_prior_order']
test_features['diff_dow'] = abs(test_features['user_product_dow']-test_features['order_dow']).map(lambda x: min (x, 7-x))
test_features['user_product_orders_since_last'] = test_features['user_orders'] - test_features['user_product_last_order_id'].map(orders.order_number)
test_features['user_product_hour_vs_last'] = abs(test_features['order_hour_of_day'] - test_features['user_product_last_order_id'].map(orders.order_hour_of_day)                                                  ).map(lambda x: min(x, 24-x))

test_features.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,user_product_id,user_product_reorder_rate,user_distinct_products,user_reordered_products,user_product_dow,user_product_hod,user_product_dsp,user_product_orders,product_id,user_product_addCart,user_avg_basket_size,user_product_last_order_id,user_reorder_rate,product_total_orders,product_reorder_rate,product_reorders,user_orders,user_order_dsp,user_total_products,diff_order_hod,ratio_dsp,diff_dow,user_product_orders_since_last,user_product_hour_vs_last
0,2774568,3,test,13,5,15,11.0,3016797,0.250000,33,19.0,0.333333,15.000000,7.000000,3,16797,4.000000,8.000000,3160850,0.575758,142951,0.698155,99802.0,12,12.090909,96.000000,0.000000,0.636364,2.333333,7.0,1.0
1,2774568,3,test,13,5,15,11.0,3047766,0.750000,33,19.0,0.888889,16.111111,12.125000,9,47766,3.777778,7.222222,1402502,0.575758,176815,0.758103,134044.0,12,12.090909,86.666667,1.111111,1.102273,2.888889,-14.0,0.0
2,2774568,3,test,13,5,15,11.0,3000248,0.083333,33,19.0,3.000000,19.000000,9.000000,1,248,3.000000,9.000000,444309,0.575758,6371,0.400251,2550.0,12,12.090909,108.000000,4.000000,0.818182,2.000000,-1.0,4.0
3,2774568,3,test,13,5,15,11.0,3001005,0.083333,33,19.0,3.000000,16.000000,17.000000,1,1005,5.000000,6.000000,676467,0.575758,463,0.440605,204.0,12,12.090909,72.000000,1.000000,1.545455,2.000000,6.0,2.0
4,2774568,3,test,13,5,15,11.0,3001819,0.250000,33,19.0,0.666667,16.333333,11.333333,3,1819,2.666667,7.333333,1839752,0.575758,2424,0.492162,1193.0,12,12.090909,88.000000,1.333333,1.030303,2.666667,-14.0,9.0


In [191]:
train_features.sort_values(['order_id','product_id'], inplace=True)
test_features.sort_values(['order_id','product_id'], inplace=True)

In [192]:
train_features.drop(['eval_set','order_number'], axis=1, inplace=True)
test_features.drop(['eval_set','order_number'], axis=1, inplace=True)

In [199]:
train_orders = orders[orders['eval_set']=='train']
trainProducts = pandas.merge(train_orders,trainingOrderItems, on='order_id')

In [200]:
trainProducts = trainProducts.groupby('user_id')['product_id'].apply(set)
trainProducts.head()

user_id
1    {196, 26405, 27845, 46149, 13032, 39657, 26088, 25133, 38928, 10258, 49235}                                                                                                                                        
2    {24838, 11913, 45066, 31883, 48523, 38547, 24852, 32792, 7963, 22559, 12324, 33957, 22825, 9387, 45613, 22963, 48821, 2361, 41787, 5699, 13640, 5450, 19019, 16589, 45645, 21329, 1757, 12007, 34284, 26352, 31612}
5    {40706, 21413, 20843, 48204, 21616, 19057, 20114, 15349, 16185}                                                                                                                                                    
7    {17638, 29894, 47272, 45066, 13198, 37999, 40852, 12053, 43967}                                                                                                                                                    
8    {27104, 15937, 5539, 41540, 31717, 48230, 22247, 41259, 37803, 21903, 10960, 7058, 4853, 47766, 48121, 25659, 23165, 28

In [195]:
trainLabels = []
for row in tqdm(train_features.itertuples()):
    trainLabels += [row.product_id in trainProducts[row.user_id]]
print(len(trainLabels))
print(train_features.shape)


8474661it [02:47, 50727.28it/s]

8474661
(8474661, 29)


In [197]:
trainLabels[:5]

[False, False, True, False, True]

In [201]:
num_feature_list=['diff_dow','ratio_dsp','diff_order_hod', 'user_avg_basket_size', 'user_product_hour_vs_last',
                  'days_since_prior_order','user_product_reorder_rate','user_product_dow','user_reordered_products',
                  'user_product_hod','user_product_dsp','user_product_orders','user_product_addCart',
                  'product_reorder_rate','product_total_orders', 'user_reorder_rate', 'user_distinct_products',
                  'user_product_orders_since_last']

In [202]:
train_features[num_feature_list] = (train_features[num_feature_list]-train_features[num_feature_list].mean())/((train_features[num_feature_list].max()-train_features[num_feature_list].min()))
test_features[num_feature_list] = (test_features[num_feature_list]-test_features[num_feature_list].mean())/((test_features[num_feature_list].max()-test_features[num_feature_list].min()))

In [203]:
train_features.head()

,order_id,user_id,order_dow,order_hour_of_day,days_since_prior_order,user_product_id,user_product_reorder_rate,user_distinct_products,user_reordered_products,user_product_dow,user_product_hod,user_product_dsp,user_product_orders,product_id,user_product_addCart,user_avg_basket_size,user_product_last_order_id,user_reorder_rate,product_total_orders,product_reorder_rate,product_reorders,user_orders,user_order_dsp,user_total_products,diff_order_hod,ratio_dsp,diff_dow,user_product_orders_since_last,user_product_hour_vs_last
4629306,1,112108,4,10,-0.178176,112108002067,0.181188,-0.140862,-0.098748,-0.292511,-0.153366,-0.189770,-0.014650,2067,-0.015349,-0.052874,2541372,0.268539,-0.040995,0.076044,1902.0,3,11.0,24.0,-0.319506,NaN,0.400300,-0.052444,-0.298248
4629308,1,112108,4,10,-0.178176,112108005707,0.517923,-0.140862,-0.098748,-0.209177,0.085764,-0.056437,-0.004446,5707,-0.029238,-0.063290,186706,0.268539,-0.045100,0.044001,690.0,3,11.0,19.5,0.138827,NaN,0.257443,-0.031931,-0.381581
4629307,1,112108,4,10,-0.178176,112108011109,0.517923,-0.140862,-0.098748,-0.292511,-0.066409,-0.189770,-0.004446,11109,-0.036182,-0.052874,2541372,0.268539,-0.038172,0.190429,3192.0,3,11.0,24.0,-0.152840,NaN,0.400300,-0.052444,-0.298248
4629304,1,112108,4,10,-0.178176,112108014947,0.854658,-0.140862,-0.098748,-0.236955,0.064025,-0.056437,0.005758,14947,-0.038497,-0.059818,186706,0.268539,0.013487,0.295131,23463.0,3,11.0,21.0,0.097160,NaN,0.305062,-0.031931,-0.381581
4629302,1,112108,4,10,-0.178176,112108022035,0.517923,-0.140862,-0.098748,-0.292511,-0.066409,-0.189770,-0.004446,22035,-0.029238,-0.052874,2541372,0.268539,0.078646,0.244622,45639.0,3,11.0,24.0,-0.152840,NaN,0.400300,-0.052444,-0.298248


In [204]:
cat_feature_list = []
feature_list = num_feature_list+cat_feature_list

In [205]:
X_train = train_features[feature_list].fillna(0).as_matrix()[:500000]
Y_train = numpy.array(trainLabels).astype('int8')[:500000]

/Users/waficel-assi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [207]:
Y_train

array([0, 0, 1, ..., 0, 0, 0], dtype=int8)

In [210]:
# fix random seed for reproducibility
numpy.random.seed(7)

# create model
model = Sequential()
model.add(Dense(13, input_dim=len(feature_list), activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [214]:
# Compile model
adam = optimizers.Adam()

# Fit the model
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=10,verbose=0, callbacks=[TQDMNotebookCallback()])


HBox(children=(IntProgress(value=0, description='Training', max=10), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 0', max=500000), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 1', max=500000), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 2', max=500000), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 3', max=500000), HTML(value='')))

...

HBox(children=(IntProgress(value=0, description='Epoch 5', max=500000), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 6', max=500000), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 7', max=500000), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 8', max=500000), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Epoch 9', max=500000), HTML(value='')))

In [215]:
X_test = test_features[feature_list].fillna(0).as_matrix()
test_preds = model.predict(X_test)


/Users/waficel-assi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [216]:
test_preds

array([[0.04546342],
       [0.09448225],
       [0.06540157],
       ...,
       [0.04896333],
       [0.12070613],
       [0.09281508]], dtype=float32)

In [217]:
past_order_id = -1
reorderedProducts = []
output = []
i =0 
maxProd = [0,0]

In [218]:
test_features.head()

,order_id,user_id,order_dow,order_hour_of_day,days_since_prior_order,user_product_id,user_product_reorder_rate,user_distinct_products,user_reordered_products,user_product_dow,user_product_hod,user_product_dsp,user_product_orders,product_id,user_product_addCart,user_avg_basket_size,user_product_last_order_id,user_reorder_rate,product_total_orders,product_reorder_rate,product_reorders,user_orders,user_order_dsp,user_total_products,diff_order_hod,ratio_dsp,diff_dow,user_product_orders_since_last,user_product_hour_vs_last
858095,17,36855,6,15,-0.446237,36855001283,0.096375,-0.145215,-0.14103,-0.458892,-0.196971,0.576189,-0.014692,1283,-0.040674,-0.061448,234692,-0.16614,-0.047506,-0.351908,6.0,4,14.0,36.000000,0.181576,NaN,-0.170464,-0.067820,-0.132405
858092,17,36855,6,15,-0.446237,36855006291,0.096375,-0.145215,-0.14103,0.541108,0.020420,-0.223811,-0.014692,6291,-0.002212,-0.061448,3248434,-0.16614,-0.043403,0.090071,1217.0,4,14.0,36.000000,-0.235091,NaN,-0.456178,-0.231922,-0.215738
858088,17,36855,6,15,-0.446237,36855007035,0.096375,-0.145215,-0.14103,-0.292225,-0.023058,NaN,-0.014692,7035,-0.059905,-0.098485,898818,-0.16614,-0.043477,-0.060709,924.0,4,14.0,20.000000,-0.151758,NaN,0.115251,-0.072948,-0.215738
858094,17,36855,6,15,-0.446237,36855011494,0.096375,-0.145215,-0.14103,-0.292225,-0.023058,NaN,-0.014692,11494,-0.040674,-0.098485,898818,-0.16614,-0.044793,-0.184154,476.0,4,14.0,20.000000,-0.151758,NaN,0.115251,-0.072948,-0.215738
858097,17,36855,6,15,-0.446237,36855013107,0.601477,-0.145215,-0.14103,0.152219,-0.066536,0.042856,0.005716,13107,-0.066315,-0.076880,1058761,-0.16614,-0.046778,0.108324,237.0,4,14.0,29.333333,-0.068424,NaN,0.210489,-0.098589,-0.132405


In [220]:
for row in tqdm(test_features.itertuples()):
    if (row.order_id!=past_order_id):
        if (past_order_id==-1):
            pass
        else:
            if (reorderedProducts == []):
                reorderedProducts.append(maxProd[0])
            output.append([past_order_id," ".join(reorderedProducts)])
            reorderedProducts = []
            maxProd = [0,0]
        past_order_id = row.order_id
        
    if (test_preds[i]>.2):
            reorderedProducts.append(str(row.product_id))
    else:
        if (test_preds[i] > maxProd[1]):
            maxProd = [str(row.product_id),test_preds[i]]
    i+=1


4833292it [00:36, 132039.65it/s]


In [221]:
output.append([past_order_id," ".join(reorderedProducts)])

output[5]

[313, '12779 13198 21903 25890 45007 46906 49683']